#선형회귀분석

##1.선형회귀분석(OLS 사용)


In [1]:
#[1] 데이터 준비

import pandas as pd

# 학습 데이터 생성
data_train = {
    '아빠': [175, 180, 172, 174, 178, 168, 173, 177],
    '엄마': [160, 158, 155, 161, 163, 160, 168, 167],
    '아들': [178, 182, 175, 180, 183, 174, 179, 183],
    '딸': [163, 168, 157, 164, 167, 158, 169, 169]
}

# 학습 데이터프레임 생성
df = pd.DataFrame(data_train)
df.index.name = 'id'  # 인덱스 이름 설정

# 필요한 컬럼을 선택하여 CSV로 저장
df[['아빠', '엄마', '아들']].to_csv('train1.csv')
df[['아빠', '엄마', '딸']].to_csv('train2.csv')

# 테스트 데이터 생성
data_test = {
    '아빠': [174, 179, 180],
    '엄마': [160, 160, 160]
}

# 테스트 데이터프레임 생성 및 저장
df2 = pd.DataFrame(data_test)
df2.index.name = 'id'
df2.to_csv('test.csv')


##2.다중선형회귀분석 수행


In [2]:
#[0] train,test 파일 가져오기
import pandas as pd
XY = pd.read_csv('train1.csv')
X_submission = pd.read_csv('test.csv')

#[1] 다중선형회귀분석
from statsmodels.api import OLS,add_constant
X = XY[['아빠','엄마']]
Y = XY['아들'] # 연속형 종속변수

# y = coef1*X['아빠'] + coef2*X['엄마'] + 절편
X = add_constant(X) #절편
display(X.head(3))
model = OLS(Y,X).fit()
#print(model.summary())
print(f'{model.rsquared:.4f},{model.rsquared_adj:.4f}')
print(model.params)

,const,아빠,엄마
0,1.0,175,160
1,1.0,180,158
2,1.0,172,155


0.9420,0.9188
const   -3.835397
아빠       0.772160
엄마       0.298743
dtype: float64


In [3]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                     아들   R-squared:                       0.942
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     40.62
Date:                Tue, 19 Nov 2024   Prob (F-statistic):           0.000809
Time:                        10:03:52   Log-Likelihood:                -9.3425
No. Observations:                   8   AIC:                             24.68
Df Residuals:                       5   BIC:                             24.92
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.8354     20.704     -0.185      0.8

/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  res = hypotest_fun_out(*samples, **kwds)


In [4]:
#[2] 예측값 구하기
model.predict([[1.0,175,160],
               [1.0,180,158]])

array([179.09144602, 182.35476109])

##3.파일 데이터를 사용한 predict 수행

In [5]:
#[4] 파일로 데이터가 주어졌는데, const 칼럼이 없는 경우
# const 컬럼을 추가하여 사용
X2 = X_submission[['아빠','엄마']]
X2.insert(0,'const',1)
model.predict(X2.values)

array([178.31928593, 182.18008636, 182.95224644])

In [6]:
#[3] 파일로 데이터가 주어졌을때, Id컬럼을 const 컬럼으로 수정하여 사용
X_submission['id'] = 1
display(X_submission.head())
model.predict(X_submission.values)

,id,아빠,엄마
0,1,174,160
1,1,179,160
2,1,180,160


array([178.31928593, 182.18008636, 182.95224644])

##4.션형회귀분석 - OLS Regression Result 항목

* Dep.Variable
  * Dependent variable,종속변수
* Model
  * 모델링 방법
* No.observations
  * Number of Observations, 총 표본의 수
* DF Resiuals
  * 잔차의 자유도,전페 표본 수에서 측정되는 변수들의 개수를 뺸것
  DF Resiuals = 표본수 - 종속변수 개수 - 독립변수 개수
* DF model
  * 독립변수의 개수
* R-sqaured
  * 결정계수, 전체 데이터 중 해당 회귀모델이 설명할 수 있는 데이터의 비율, 회귀식의 설명력을 나타낸다
* Adf.R-squared
  * 조정된 결정계수
* F-statistic
  * F 통계량
* Prob(F-statistic)
  * 회귀식이 유의미한지 판단 (0.05이하일 경우 변수끼리 매우 관련 있다고 판단)
* AIC
  * 표본의 개수와 모델의 복잡성을 기반으로 모델을 평가하며, 수치가 낮을수록 좋음
* BIC
  * AIC와 유사하나 패널티를 부여하여 AIC보다 모델 평가 성능이 더 좋으며, 수치가 낮을 수록 좋음
* coef
  * 회귀계수
* std err
  * 계수 추정치의 표준오차(standarad error),값이 작을 수록 좋음
* t
  * t-test, 독립년수와 종속변수 사이의 상관관계, 값이 클수록 상관도가 큼
* p-value(P>|t|)
  * 독립년수들의 유의확률 , 0.05보다 작아야 유의미함
* [0.025,0.975]
  * 회귀계수의 신뢰구간
* Omnibus
  * 디아고스티노 검정, 비대칭도와 첨도를 결합한 정규성 테스트이며 값이 클 수록 정규분포를 따름
* Prob(Omnibus)
  * 디아고스티노 검정이 유의미한지 판단(0.05 이하일 경우 유의미하다고 판단)
* Skew(왜도)
  * 평균 주위에 잔차들의 대칭하는지를 보는것이며, 0에 가까울수록 대칭이다.
* Kurtosis(첨도)
  * 잔차들의 분포 모양이며, 3에 가까울수록 정규 분포이다,(음수는 평평한 형태, 양수는 뾰족한 형태)
* Durbin-Watson
  * 더빈-왓슨 정규성 검정이며, 잔차의 독립성 여부를 판단(1.5~2.5 사이일때 잔차는 독립적이라고 판단하여 0이나 4에 가까울수록 잔차들은 자기상관을 가지고 있다고 판단)
* Jarque Bera(JB)
  * 자크베라 정규성 검정, 값이 클수록 정규분포의 데이터를 사용했다는 것
* Cond.NO
  * 다중공선성 검정, 독립변수간 상관간계가 있는지 보는것이며, 10이상이면 다중공선성이 있다고 판다.

##5.아이스티주문 예측 encoding이 필요한 경우 - OLS 사용

* 종속변수
  * order
* 독립변수
  * weekday,highigh_temperature

In [7]:
from statsmodels.api import OLS,add_constant
from sklearn.preprocessing import LabelEncoder
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/jmnote/zdata/master/simple-regression/iced-tea-orders.csv')
display(df.head(3))
encoder = LabelEncoder()
df['weekday'] = encoder.fit_transform(df['weekday'])
display(df.head(3))
X = df[['weekday','high_temperature']]
Y = df['order']
X = add_constant(X)
model = OLS(Y,X).fit()
print(model.params)

,date,weekday,high_temperature,order
0,2002-07-22,Mon,29,77
1,2002-07-23,Tue,28,62
2,2002-07-24,Wed,34,93


,date,weekday,high_temperature,order
0,2002-07-22,1,29,77
1,2002-07-23,5,28,62
2,2002-07-24,6,34,93


const              -35.761795
weekday              0.574146
high_temperature     3.658213
dtype: float64


In [8]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  order   R-squared:                       0.831
Model:                            OLS   Adj. R-squared:                  0.800
Method:                 Least Squares   F-statistic:                     26.95
Date:                Tue, 19 Nov 2024   Prob (F-statistic):           5.76e-05
Time:                        10:03:54   Log-Likelihood:                -42.851
No. Observations:                  14   AIC:                             91.70
Df Residuals:                      11   BIC:                             93.62
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const              -35.7618     15.014  

/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  res = hypotest_fun_out(*samples, **kwds)
